<a href="https://colab.research.google.com/github/kavyajeetbora/foursquare_ai/blob/master/notebooks/12_tilemaker_india_pbf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tilemaker

1. [Creating tiles from pbf files using `tilemaker`](https://github.com/systemed/tilemaker)

2.

# Setup Environment

1. Install Pmtiles
2. GDAL
3. tilemaker

In [ ]:
import requests
import zipfile
import io
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon

# ----------------------------
# Download Natural Earth World
# ----------------------------
def download_naturalearth_world():
    url = "https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip"
    r = requests.get(url)
    r.raise_for_status()

    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall("naturalearth_world")  # extracts shapefiles to folder

    # find .shp path
    for f in z.namelist():
        if f.endswith(".shp"):
            return "naturalearth_world/" + f

    raise RuntimeError("Shapefile not found in Natural Earth zip")

# ----------------------------
# Download INDIA poly
# ----------------------------
def download_poly(url, filename="india.poly"):
    r = requests.get(url)
    r.raise_for_status()
    with open(filename, "wb") as f:
        f.write(r.content)
    return filename

# ----------------------------
# Parse .poly into shapely MultiPolygon
# ----------------------------
def parse_poly_to_multipolygon(filename):
    polygons = []
    current = []

    with open(filename, "r") as f:
        lines = f.read().strip().splitlines()

    i = 1  # skip name line

    while i < len(lines):
        line = lines[i].strip()

        if line.isdigit():
            i += 1
            ring = []
            while i < len(lines) and lines[i].strip() != "END":
                x, y = map(float, lines[i].split())
                ring.append((x, y))
                i += 1
            current.append(ring)
        elif line == "END":
            if current:
                polygons.append(current)
                current = []
            i += 1
        else:
            i += 1

    shapely_polys = []
    for rings in polygons:
        outer = rings[0]
        holes = rings[1:] if len(rings) > 1 else None
        shapely_polys.append(Polygon(outer, holes))

    return MultiPolygon(shapely_polys)

# ----------------------------
# Export GeoJSON
# ----------------------------
def save_geojson(geom, filename):
    gpd.GeoDataFrame({"geometry": [geom]}, crs="EPSG:4326").to_file(filename, driver="GeoJSON")
    return filename

# ----------------------------
# PIPELINE
# ----------------------------
# Step 1: world basemap
world_shp = download_naturalearth_world()
world = gpd.read_file(world_shp)

# Step 2: union world geometry
world_union = world.dissolve().geometry.iloc[0]

# Step 3: India boundary
poly_file = download_poly("https://download.geofabrik.de/asia/india.poly")
india_geom = parse_poly_to_multipolygon(poly_file)

# Step 4: subtract India
masked_world = world_union.difference(india_geom)

# Step 5: save result
save_geojson(masked_world, "world_minus_india.geojson")

print("Saved world_minus_india.geojson")

In [ ]:
%%shell

# Download go-pmtiles for Linux x86_64 quietly
wget -q https://github.com/protomaps/go-pmtiles/releases/download/v1.28.3/go-pmtiles_1.28.3_Linux_x86_64.tar.gz

# Verify SHA256 checksum
echo "06cf492adc2c7fcd23c4f11a98a5292f4cbe04d3afc3a6b38a07bb47452daca2 go-pmtiles_1.28.3_Linux_x86_64.tar.gz" | sha256sum --check --quiet

# Extract quietly
tar -xzf go-pmtiles_1.28.3_Linux_x86_64.tar.gz -C /tmp/ > /dev/null 2>&1

# Install binary to /usr/local/bin (assuming binary is named 'pmtiles')
sudo mv /tmp/pmtiles /usr/local/bin/pmtiles > /dev/null 2>&1

# Clean up
rm go-pmtiles_1.28.3_Linux_x86_64.tar.gz

# Check installation
echo "pmtiles Installation complete."
pmtiles version

## Installing `tilemaker`

In [ ]:
%%shell
set -euo pipefail
export DEBIAN_FRONTEND=noninteractive

apt-get update -qq
apt-get install -y -qq \
  git build-essential \
  libboost-dev libboost-filesystem-dev libboost-program-options-dev \
  libsqlite3-dev rapidjson-dev zlib1g-dev libshp-dev \
  lua5.3 liblua5.3-dev \
  protobuf-compiler libprotobuf-dev

cd /content
if [ ! -d tilemaker ]; then
  git clone --depth 1 https://github.com/systemed/tilemaker.git
fi

cd tilemaker
make -j"$(nproc)"
sudo make install

## Download OSM data for India

In [ ]:
%%shell
cd /content
wget -O india.osm.pbf https://download.geofabrik.de/asia/india-latest.osm.pbf

In [ ]:
%%shell
sudo apt install osmium-tool

## Filter OSM Data for Places only

In [ ]:
%%shell
osmium tags-filter india.osm.pbf nw/place r/place -o india_places.osm.pbf

In [ ]:
# %%shell
# sed -i 's/name:latin/name:en/g' /content/tilemaker/resources/process-openmaptiles.lua

In [ ]:
%%shell
tilemaker /content/india_places.osm.pbf --output /content/india-latest-en.mbtiles --config /content/tilemaker/resources/config-openmaptiles.json --process /content/tilemaker/resources/process-openmaptiles.lua

In [ ]:
from google.colab import files
files.download('/content/india-latest-en.mbtiles')

Convert mbtiles to pmtiles

In [ ]:
# %%shell
# pmtiles convert india-latest.mbtiles india-latest.pmtiles

In [ ]:
# from google.colab import files
# files.download('/content/india-latest.pmtiles')